In [1]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt 
import seaborn as sns
from IPython.display import display

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.float_format", "{:.4f}".format)

input_data_path = "../data/input_data.xlsx"
final_data_path = "../data/final_data.xlsx"
imputed_data_path = "../data/imputed_data.xlsx"
nutrition_data_path = "../data/nutrition_data.xlsx"

input_df = pd.ExcelFile(input_data_path).parse(sheet_name="Sheet1")
final_df = pd.ExcelFile(final_data_path).parse(sheet_name="Sheet2")
imputed_df = pd.ExcelFile(imputed_data_path).parse(sheet_name="Sheet1") # (341, 38)
nutrition_df = pd.ExcelFile(nutrition_data_path).parse(sheet_name="Sheet1") # (2191, 64)

In [2]:
# GẮN CỘT NUMBER VÀO IMPUTED DATA

imputed_df.insert(0, "sub_id", final_df["number"])

In [3]:
# LỌC BỎ PHẦN TPLS_

nutrition_df.loc[nutrition_df["Number"].str.contains(r"^TPLS_", case=False, na=False), 
    "Number"
] = nutrition_df["Number"].str.replace(r"^TPLS_", "", regex=True)

In [4]:
# KIỂM TRA MỘT SỐ CỘT NGHI NGỜ 

# test_cols = ["sedentary_lifestyle_hour_day", "PSQI_score", "snoring_times_week", "sleep_apnea_times_week"]

# test_cols_1 = ["CPP", "GDM", "preterm_birth", "education_level", "mother_diabetes", "father_diabetes", "pregnancy_smoking"]

# for col in test_cols_1:
#     display(imputed_df[col].value_counts())

In [5]:
# CHUẨN BỊ IMPUTED_SUBSET

imputed_df.rename(columns={"cholesterol": "cholesterol_blood"}, inplace=True)

removed_cols = [
    # đây có thể là mediator, không phải là exposure
    "waist", "hip", "SBP", "DBP",
    # các biến số học có số giá trị không đáng tin cậy, gây lỗi và khó khăn khi định dạng biến danh định hay số học
    "snoring_times_week", "sleep_apnea_times_week",
    # các biến chỉ có 1 cá nhân thuộc về một giá trị, khiến mô hình gặp lỗi
    "pregnancy_smoking"
]

imputed_subset = imputed_df.drop(columns=removed_cols).copy()

imputed_subset["insulin_re"] = imputed_subset.apply(
    lambda x: 0 if (x["insulin"] * x["glucose_ac"]) / 405 <= 3.16 else 1,
    axis=1
)

imputed_subset.drop(columns=["insulin", "glucose_ac"], inplace=True)

imputed_subset.columns

Index(['sub_id', 'age', 'zbmi', 'cholesterol_blood', 'TG', 'HDL', 'cortisol',
       'CPP', 'sex', 'family_income', 'GDM', 'gestational_weight_gain',
       'preterm_birth', 'gestational_age_week', 'birth_weight_gram',
       'exclusive_breastfeeding_month', 'mixed_breastfeeding_month',
       'father_diabetes', 'mother_diabetes', 'education_level', 'father_BMI',
       'mother_BMI', 'sedentary_lifestyle_hour_day',
       'low_physical_activity_hour_day', 'sleep_duration', 'PSQI_score',
       'score_5A', 'PSS', 'score_5B', 'CES_D', 'score_5C', 'RSE',
       'insulin_re'],
      dtype='object')

In [6]:
# ĐÁNH GIÁ MỨC ĐỘ THIẾU DỮ LIỆU DINH DƯỠNG 

nutrition_cols = nutrition_df.columns

high_missing_cols = []

for col in nutrition_cols: 
    missing_percent = (nutrition_df[col].isna().mean() * 100).round(2)
    if missing_percent > 40:
        high_missing_cols.append(col)
    # print(f"The {col} column has a missing data rate of {missing_percent}%.")


In [7]:
# ĐIỀN GIÁ TRỊ THIẾU TRONG DỮ LIỆU DINH DƯỠNG BẰNG MEDIAN 

nutrition_df = nutrition_df.fillna(nutrition_df.median(numeric_only=True))

# chú ý cách lập luận tại sao lại chọn điền bằng median

In [8]:
# ĐỔI TÊN BẢNG DỮ LIỆU DINH DƯỠNG 

column_names_map = {
    "Number": "sub_id",
    "Energy(kcal)": "energy_kcal", 
    "Crude_Protein(g)": "crude_protein_g", 
    "Crude_Fat(g)": "crude_fat_g", 
    "Total_Carbohydrate(g)": "total_carbohydrate_g", 
    "Water(g)": "water_g", 
    "Fruits": "fruits", 
    "Vegetables": "vegetables", 
    "WholeGrainsAndRoots": "whole_grains_roots", 
    "protein(LowFat)": "protein_low_fat", 
    "protein(MediumFat)": "protein_medium_fat", 
    "protein(HighFat)": "protein_high_fat", 
    "protein(SuperHighFat)": "protein_superhigh_fat", 
    "Dairy(Skim)": "dairy_skim", 
    "Dairy(LowFat)": "dairy_low_fat", 
    "Dairy(WholeFat)": "dairy_whole_fat", 
    "Oils_Nuts_And_Seeds": "oils_nuts_and_seeds", 
    "Total_Polyunsaturated_Fatty_Acids(g)": "total_polyunsaturated_fatty_acids_g", 
    "Total_Monounsaturated_Fatty_Acids(g)": "total_monounsaturated_fatty_acids_g", 
    "Total_Saturated_Fatty_Acids(g)": "total_saturated_fatty_acids_g", 
    "CrudeFiber(g)": "crude_fiber_g", 
    "Dietary_Fiber(g)": "dietary_fiber_g", 
    "Total_Sugar(g)": "total_sugar_g", 
    "Glucose(g)": "glucose_g", 
    "Fructose(g)": "fructose_g", 
    "Maltose(g)": "maltose_g", 
    "Sucrose(g)": "sucrose_g",
    "Lactose(g)": "lactose_g", 
    "Cholesterol(mg)": "cholesterol_mg", 
    "Sodium(mg)": "sodium_mg", 
    "Potassium(mg)": "potassium_mg", 
    "Calcium(mg)": "calcium_mg", 
    "Magnesium(mg)": "magnesium_mg", 
    "Phosphorus(mg)": "phosphorus_mg", 
    "Iron(mg)": "iron_mg", 
    "Zinc(mg)": "zinc_mg", 
    "VitaminB1(Thiamin)(mg)": "vitamin_b1_mg", 
    "VitaminB2(Riboflavin)(mg)": "vitamin_b2_mg", 
    "Niacin(mg)": "niacin_mg", 
    "VitaminB6(mg)": "vitamin_b6_mg", 
    "VitaminB12(ug)": "vitamin_b12_ug", 
    "FolicAcid(ug)": "folicacid_ug", 
    "VitaminC(mg)": "vitaminc_mg",
    "TotalVitaminA(I.U.)": "totalVitaminA_ui", 
    "TotalVitaminE(mg)": "totalVitaminE_mg", 
    "trans_fat": "trans_fat", 
    "Threonine(mg)": "threonine_mg", 
    "Valine(mg)": "valine_mg", 
    "Methionine(mg)": "methionine_mg", 
    "Isoleucine(mg)": "isoleucine_mg", 
    "Leucine(mg)": "leucine_mg", 
    "Phenylalanine(mg)": "phenylalanine_mg", 
    "Lysine(mg)": "lysine_mg", 
    "Histidine(mg)": "histidine_mg", 
    "Tryptophan(mg)": "tryptophan_mg", 
    "Alanine(mg)": "alanine_mg", 
    "Arginine(mg)": "arginine_mg", 
    "AsparticAcid(mg)": "aspartic_mg", 
    "GlutamicAcid(mg)": "glutamic_mg", 
    "Glycine(mg)": "glycine_mg", 
    "Proline(mg)": "proline_mg", 
    "Serine(mg)": "serine_mg", 
    "Tyrosine(mg)": "tyrosine_mg", 
    "Cystine(mg)": "cystine_mg"           
}

nutrition_df.rename(columns=column_names_map, inplace=True)

# len(column_names_map) # 63 không tính cột number

In [9]:
# MỘT SỐ CỘT CÓ VẤN ĐỀ

nutrition_df.drop(columns=["total_sugar_g", "crude_fiber_g"], inplace=True) # hai cột trên không có trong biểu đồ tròn
# biến Cu (Copper) có trong biểu đồ tròn nhưng không có trong cơ sở dữ liệu 
# biến calories trong biểu đồ tròn lấy giá trị của cột nào: cột energy_kcal trong nutrition_df hay cột calories(kcal) trong imputed_subset


In [10]:
# GỘP MỘT SỐ CỘT ĐỂ THÀNH FAMILY 

# EAA 
nutrition_df["EAA"] = nutrition_df["threonine_mg"] + nutrition_df["valine_mg"] + nutrition_df["methionine_mg"] + nutrition_df["isoleucine_mg"] + nutrition_df["leucine_mg"] + nutrition_df["phenylalanine_mg"] + nutrition_df["lysine_mg"] + nutrition_df["histidine_mg"] + nutrition_df["tryptophan_mg"] 

# NEAA
nutrition_df["NEAA"] = nutrition_df["alanine_mg"] + nutrition_df["arginine_mg"] + nutrition_df["aspartic_mg"] + nutrition_df["glutamic_mg"] + nutrition_df["glycine_mg"] + nutrition_df["proline_mg"] + nutrition_df["serine_mg"] + nutrition_df["tyrosine_mg"] + nutrition_df["cystine_mg"]

# PLMF
nutrition_df["PLMF"] = nutrition_df["protein_low_fat"] + nutrition_df["protein_medium_fat"]

# PHSF 
nutrition_df["PHSF"] = nutrition_df["protein_high_fat"] + nutrition_df["protein_superhigh_fat"]

# monosaccharides
nutrition_df["monosaccharides"] = nutrition_df["glucose_g"] + nutrition_df["fructose_g"]

# disaccharides
nutrition_df["disaccharides"] = nutrition_df["maltose_g"] + nutrition_df["sucrose_g"] + nutrition_df["lactose_g"]

# dairy (skim & lowfat)
nutrition_df["dairy_skim_low_fat"] = nutrition_df["dairy_skim"] + nutrition_df["dairy_low_fat"]

# VitB
nutrition_df["totalVitaminB_mg"] = nutrition_df["vitamin_b1_mg"] + nutrition_df["vitamin_b2_mg"] + nutrition_df["niacin_mg"] + nutrition_df["vitamin_b6_mg"] + nutrition_df["vitamin_b12_ug"] + nutrition_df["folicacid_ug"] 

In [11]:
# CHUẨN BỊ NUTRI_SUBSET

nutri_subset = nutrition_df[[
    "sub_id",
    "energy_kcal",
    # nhóm protein 
    "crude_protein_g", 
    "EAA", 
    "NEAA", 
    "PLMF", 
    "PHSF", 
    # nhóm chất béo 
    "crude_fat_g", 
    "trans_fat", 
    "total_saturated_fatty_acids_g", 
    "total_monounsaturated_fatty_acids_g", 
    "total_polyunsaturated_fatty_acids_g", 
    "cholesterol_mg", 
    "oils_nuts_and_seeds", 
    # nhóm CH
    "total_carbohydrate_g", 
    "whole_grains_roots", 
    "monosaccharides", 
    "disaccharides", 
    # nhóm nước, trái cây, rau xanh
    "water_g", 
    "fruits", 
    "vegetables", 
    "dietary_fiber_g",
    # nhóm sữa
    "dairy_whole_fat", 
    "dairy_skim_low_fat", 
    # nhóm ions 
    "sodium_mg",
    "potassium_mg",
    "calcium_mg", 
    "magnesium_mg", 
    "phosphorus_mg",
    "iron_mg",
    "zinc_mg",
    # nhóm vitamin 
    "totalVitaminA_ui", 
    "totalVitaminB_mg",
    "vitaminc_mg", 
    "totalVitaminE_mg"
]].copy()

In [12]:
# ĐỔI TÊN CỘT 

nutri_subset_names_mapper = {
    "energy_kcal": "calories",
    # nhóm protein 
    "crude_protein_g": "crude_protein", 
    "EAA": "EAA", 
    "NEAA": "NEAA", 
    "PLMF": "PLMF", 
    "PHSF": "PHSF", 
    # nhóm chất béo 
    "crude_fat_g": "crude_fat", 
    "trans_fat": "trans_fat", 
    "total_saturated_fatty_acids_g": "TFA_S", 
    "total_monounsaturated_fatty_acids_g": "TFA_M", 
    "total_polyunsaturated_fatty_acids_g": "TFA_P", 
    "cholesterol_mg": "cholesterol", 
    "oils_nuts_and_seeds": "seeds", 
    # nhóm CH
    "total_carbohydrate_g": "total_carbohydrate", 
    "whole_grains_roots": "whole_rhizome", 
    "monosaccharides": "monosaccharides", 
    "disaccharides": "disaccharides", 
    # nhóm nước, trái cây, rau xanh
    "water_g": "water", 
    "fruits": "fruit", 
    "vegetables": "vegetable", 
    "dietary_fiber_g": "dietary_fiber",
    # nhóm sữa
    "dairy_whole_fat": "dairy_fullfat", 
    "dairy_skim_low_fat": "dairy_skim_lowfat", 
    # nhóm ions 
    "sodium_mg": "sodium",
    "potassium_mg": "potassium",
    "calcium_mg": "calcium", 
    "magnesium_mg": "magnesium", 
    "phosphorus_mg": "phosphorus",
    "iron_mg": "iron",
    "zinc_mg": "zinc",
    # nhóm vitamin 
    "totalVitaminA_ui": "vitA", 
    "totalVitaminB_mg": "vitB", 
    "vitaminc_mg": "vitC",
    "totalVitaminE_mg": "vitE"
}

nutri_subset.rename(columns=nutri_subset_names_mapper, inplace=True)



In [13]:
# MERGE DATA

merged_df = pd.merge(imputed_subset, nutri_subset, how="inner", on="sub_id").reset_index(drop=True)

In [14]:
# BIẾN ĐỔI NHÃN CÁC BIẾN SỐ DANH ĐỊNH 

# GDM_code = {"no-dia": 1, "gmd": 2, "other-dia": 3}
GDM_code = {1: "0", 2: "1", 3: "0"}
# education_level_code = {"senior high": 1, "vocational": 2, "graduate": 3, "college": 4}
education_level_code = {1: "1", 2: "1", 3: "3", 4: "2"}

merged_df["GDM"] = merged_df["GDM"].map(GDM_code).astype("int")
merged_df["education_level"] = merged_df["education_level"].map(education_level_code).astype("int")

In [15]:
# IMPUTATION CHO CÁC CỘT CES_D, PSS, RSE
# Đã mọi cách không thể được, nên đành điền khuyết giá trị median 

merged_df = merged_df.fillna(merged_df.median(numeric_only=True))

# merged_df.isna().mean()

In [16]:
# XUẤT FILE MERGED DATA

# merged_df.to_csv("../data/exposome_data/total_exwas_data.csv", sep=",", index=False, encoding="utf-8")

In [17]:
# CÁC CỘT LÀ BIẾN DANH ĐỊNH / CONFOUNDER

cat_cols = ["sub_id", "CPP", "sex", "GDM", "preterm_birth", "father_diabetes", "mother_diabetes", "education_level", "insulin_re"]
confounder_cols = ["age", "zbmi"]

In [23]:
# BÌNH THƯỜNG HÓA VÀ CHUẨN HÓA DỮ LIỆU 

from scipy import stats
from sklearn.preprocessing import StandardScaler
import warnings

trans_cols = merged_df.drop(columns=[*cat_cols, *confounder_cols]).copy().columns.to_list()

def boxcox_zscore(
    df, 
    cols=None, 
    shift_positive=True, 
    min_positive_shift=1e-6,
    return_params=False
):
    """
    Áp dụng Box-Cox transformation + Z-score standardization.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Dữ liệu đầu vào
    cols : list, optional
        Danh sách cột cần transform. Nếu None → tất cả numeric > 0
    shift_positive : bool
        Nếu True → tự động shift dữ liệu <= 0 lên dương
    min_positive_shift : float
        Giá trị nhỏ nhất để shift (tránh chia 0)
    return_params : bool
        Nếu True → trả thêm lambda và scaler
    
    Returns:
    --------
    df_norm : pd.DataFrame
        Dữ liệu đã Box-Cox + Z-score
    (lambda_dict, scaler) : nếu return_params=True
    """
    df = df.copy()
    if cols is None:
        # Tự động chọn cột numeric và có giá trị > 0
        cols = df.select_dtypes(include=[np.number]).columns.tolist()
        cols = [c for c in cols if (df[c] > 0).all()]
    
    lambda_dict = {}
    transformed_cols = []
    
    for col in cols:
        x = df[col].copy()
        
        # Bước 1: Đảm bảo dữ liệu > 0
        if (x <= 0).any():
            if shift_positive:
                shift = abs(x.min()) + min_positive_shift if x.min() <= 0 else 0
                x = x + shift
                lambda_dict[col] = {'shift': shift}
            else:
                warnings.warn(f"Cột {col} có giá trị <= 0 → bỏ qua Box-Cox")
                transformed_cols.append(x.values.reshape(-1, 1))
                continue
        
        # Bước 2: Box-Cox
        try:
            x_bc, lmbda = stats.boxcox(x)
            df[col] = x_bc
            lambda_dict[col] = {'lambda': lmbda, 'shift': lambda_dict.get(col, {}).get('shift', 0)}
            transformed_cols.append(x_bc.reshape(-1, 1))
        except:
            warnings.warn(f"Box-Cox thất bại với cột {col}")
            continue
    
    # Bước 3: Z-score standardization
    if transformed_cols:
        scaler = StandardScaler()
        normalized = scaler.fit_transform(np.hstack(transformed_cols))
        normalized_df = pd.DataFrame(normalized, columns=cols, index=df.index)
        df.update(normalized_df)
    
    if return_params:
        return df, lambda_dict, scaler
    else:
        return df

trans_df = boxcox_zscore(
    df = merged_df, 
    cols = trans_cols
)

trans_cols

['cholesterol_blood',
 'TG',
 'HDL',
 'cortisol',
 'family_income',
 'gestational_weight_gain',
 'gestational_age_week',
 'birth_weight_gram',
 'exclusive_breastfeeding_month',
 'mixed_breastfeeding_month',
 'father_BMI',
 'mother_BMI',
 'sedentary_lifestyle_hour_day',
 'low_physical_activity_hour_day',
 'sleep_duration',
 'PSQI_score',
 'score_5A',
 'PSS',
 'score_5B',
 'CES_D',
 'score_5C',
 'RSE',
 'calories',
 'crude_protein',
 'EAA',
 'NEAA',
 'PLMF',
 'PHSF',
 'crude_fat',
 'trans_fat',
 'TFA_S',
 'TFA_M',
 'TFA_P',
 'cholesterol',
 'seeds',
 'total_carbohydrate',
 'whole_rhizome',
 'monosaccharides',
 'disaccharides',
 'water',
 'fruit',
 'vegetable',
 'dietary_fiber',
 'dairy_fullfat',
 'dairy_skim_lowfat',
 'sodium',
 'potassium',
 'calcium',
 'magnesium',
 'phosphorus',
 'iron',
 'zinc',
 'vitA',
 'vitB',
 'vitC',
 'vitE']

In [19]:
# TẠO FILE EXPOSURES

phenotype_cols_0 = ['age', 'zbmi', 'sex', 'calories', 'insulin_re']
phenotype_cols_1 = ['age', 'zbmi', 'sex', 'insulin_re']

# exposures_df_0 không có calories
exposures_df_0 = trans_df.drop(columns=phenotype_cols_0).copy()

# exposures_df_1 có calories
exposures_df_1 = trans_df.drop(columns=phenotype_cols_1).copy()

# exposures_df.shape # (237, 66)

# exposures_df_0.to_csv("../data/exposome_data/exposures_0.csv", sep=",", index=False, encoding="utf-8")
# exposures_df_1.to_csv("../data/exposome_data/exposures_1.csv", sep=",", index=False, encoding="utf-8")

In [ ]:
# TẠO FILE PHENOTYPES

phenotypes_df = merged_df[["sub_id", *phenotype_cols_0]].copy()

# phenotypes_df.shape # (237, 5)

# phenotypes_df.to_csv("../data/exposome_data/phenotypes.csv", sep=",", index=False, encoding="utf-8")

In [ ]:
# TẠO FILE DESCRIPTION 

families_data_0 = {
    'Prenatal': ['GDM', 'gestational_weight_gain'],
    'Perinatal': ['preterm_birth', 'gestational_age_week', 'birth_weight_gram'],
    'Postnatal': ['CPP', 'exclusive_breastfeeding_month', 'mixed_breastfeeding_month'],
    'Biomaker': ['cholesterol_blood', 'TG', 'HDL', 'cortisol'],
    'Family': ['family_income', 'father_diabetes', 'mother_diabetes', 'education_level', 'father_BMI', 'mother_BMI'],
    'Activity': ['sedentary_lifestyle_hour_day', 'low_physical_activity_hour_day'],
    'Sleep': ['sleep_duration', 'PSQI_score'],
    'Psycho': ['score_5A', 'PSS', 'score_5B', 'CES_D', 'score_5C', 'RSE'],
    'Protein': ['crude_protein', 'EAA', 'NEAA', 'PLMF', 'PHSF'],
    'Fat': ['crude_fat', 'trans_fat', 'TFA_S', 'TFA_M', 'TFA_P', 'cholesterol', 'seeds'],
    'Carbohydrates': ['total_carbohydrate', 'whole_rhizome', 'monosaccharides', 'disaccharides'],
    'Water fruits vegetables': ['water', 'fruit', 'vegetable', 'dietary_fiber'],
    'Diary': ['dairy_fullfat', 'dairy_skim_lowfat'],
    'Ions': ['sodium', 'potassium', 'calcium', 'magnesium', 'phosphorus', 'iron', 'zinc'],
    'Vitamins': ['vitA', 'vitB', 'vitC', 'vitE']
}

families_data_1 = {
    'Prenatal': ['GDM', 'gestational_weight_gain'],
    'Perinatal': ['preterm_birth', 'gestational_age_week', 'birth_weight_gram'],
    'Postnatal': ['CPP', 'exclusive_breastfeeding_month', 'mixed_breastfeeding_month'],
    'Biomaker': ['cholesterol_blood', 'TG', 'HDL', 'cortisol'],
    'Family': ['family_income', 'father_diabetes', 'mother_diabetes', 'education_level', 'father_BMI', 'mother_BMI'],
    'Activity': ['sedentary_lifestyle_hour_day', 'low_physical_activity_hour_day'],
    'Sleep': ['sleep_duration', 'PSQI_score'],
    'Psycho': ['score_5A', 'PSS', 'score_5B', 'CES_D', 'score_5C', 'RSE'],
    'Calories': ['calories'],
    'Protein': ['crude_protein', 'EAA', 'NEAA', 'PLMF', 'PHSF'],
    'Fat': ['crude_fat', 'trans_fat', 'TFA_S', 'TFA_M', 'TFA_P', 'cholesterol', 'seeds'],
    'Carbohydrates': ['total_carbohydrate', 'whole_rhizome', 'monosaccharides', 'disaccharides'],
    'Water fruits vegetables': ['water', 'fruit', 'vegetable', 'dietary_fiber'],
    'Diary': ['dairy_fullfat', 'dairy_skim_lowfat'],
    'Ions': ['sodium', 'potassium', 'calcium', 'magnesium', 'phosphorus', 'iron', 'zinc'],
    'Vitamins': ['vitA', 'vitB', 'vitC', 'vitE']
}

records_0 = []

for family_name, exposures in families_data_0.items():
    for exposure in exposures:
        records_0.append({'exposure': exposure, 'family': family_name})

families_df_0 = pd.DataFrame(records_0)

# families_df_0.to_csv("../data/exposome_data/description_0.csv", sep=",", index=False, encoding="utf-8")

records_1 = []

for family_name, exposures in families_data_1.items():
    for exposure in exposures:
        records_1.append({'exposure': exposure, 'family': family_name})

families_df_1 = pd.DataFrame(records_1)

# families_df_1.to_csv("../data/exposome_data/description_1.csv", sep=",", index=False, encoding="utf-8")



sub_id,birth_weight_gram,Calcium,Calories,CES_D,cortisol,CPP,Crude fat,Crude protein,Cholesterol,cholesterol_blood, Dairy fullfat,Dairy skim & lowfat,Dietary fiber,Disaccharides,EAA,education_level,exclusive_breastfeeding_month,family_income,father_BMI,father_diabetes,Fruit,GDM,gestational_age_week,gestational_weight_gain,HDL,Iron,low_physical_activity_hour_day,Magnesium,mixed_breastfeeding_month,Monosaccharides,mother_BMI,mother_diabetes,NEAA,PLMF,Potassium,pregnancy_smoking,preterm_birth,PSQI_score,PSS,Phosphorus,PHSF,RSE,score_5A,score_5B,score_5C,sedentary_lifestyle_hour_day,Seeds,sleep_apnea_times_week,sleep_duration,snoring_times_week,Sodium,TFA_M,TFA_P,TFA_S,TG,Total carbohydrate,Trans fat,Vegetable,VitA,VitB,VitC,VitE,Water,Whole rhizome,Zinc